In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, when, col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

import plotly.express as px
import xarray as xr

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt
import os

In [ ]:
bank_df = pd.read_csv("D:\\downl\\final\\Loan_default.csv")

In [ ]:
bank_df.head()

In [ ]:
bank_df.shape

In [ ]:
bank_df.info()

In [ ]:
bank_df.describe().transpose()

In [ ]:
bank_df.isnull().sum()

In [ ]:
avg_age = bank_df["Age"].mean()
print ("The average age of this dataset is {:.1f}.".format(avg_age))

In [ ]:
# Building a spark session
findspark.init()
spark = SparkSession.builder.appName("bank_info").getOrCreate()

In [ ]:
path = "Loan_default.csv"

In [ ]:
bank_df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
bank_df.show()

In [ ]:
num_rows = bank_df.count()
num_columns = bank_df.columns
print("Number of rows:", num_rows)
print("Number of columns:", len(num_columns))

Below shows a hierarchical representation of the DataFrame's schema. Each line represents a column, showing its name and data type. 

In [ ]:
bank_df.printSchema()

In [ ]:
# Create a new DataFrame with the correct colums as a float datatype. 
columns_to_cast = ["Age", "Income", "LoanAmount", "CreditScore", "MonthsEmployed", "NumCreditLines", "InterestRate", "LoanTerm", "DTIRatio","Default"]

for i in columns_to_cast:
    bank_df = bank_df.withColumn(i, col(i).cast('float'))

In [ ]:
bank_df.printSchema()

In [ ]:
bank_df.createOrReplaceTempView("bank_table")

In [ ]:
# Calculate average DTIRatio
avg_dti_result = spark.sql("SELECT AVG(DTIRatio) as avg_dti FROM bank_table")
avg_dti = avg_dti_result.first()["avg_dti"]

# Calculate average Default
avg_default_result = spark.sql("SELECT AVG(Default) as avg_default FROM bank_table")
avg_default = avg_default_result.first()["avg_default"]

# Count records with Default=1
count_default_df_result = spark.sql("SELECT COUNT(*) as count_default FROM bank_table WHERE Default = 1")
count_default_df = count_default_df_result.first()["count_default"]

# Count total records
bank_count_result = spark.sql("SELECT COUNT(*) as bank_count FROM bank_table")
bank_count = bank_count_result.first()["bank_count"]

# Display the results
print(f"The total number of records in this dataset is: {bank_count}")
print(f"The average Debt to income ratio across the data set is: {avg_dti}")
print(f"The average default rate is: {avg_default}, meaning of the {bank_count} records {count_default_df} defaulted.")

In [ ]:
# Create a table showing the average Debt-To-Income Ratio and Default
query = """
SELECT COUNT(*) AS TotalRecords, AVG(DTIRatio) AS AVGDTIRatio, AVG(Default) AS AVGDefault
FROM bank_table
"""
spark.sql(query).show()

In [ ]:
# SQL query to select ages over 55 and their interest rate
query = """
SELECT Age, InterestRate
FROM bank_table
WHERE Age > 55
"""


spark.sql(query).show()

In [ ]:
# Average Interest Rate for people over 55
query = """
SELECT AVG(InterestRate) AS AvgInterestRateOver55
FROM bank_table
WHERE Age > 55
"""
spark.sql(query).show()

In [ ]:
# Average Interest Rate for people under 55
query = """
SELECT AVG(InterestRate) AS AvgInterestRateUnder55
FROM bank_table
WHERE Age < 55
"""
spark.sql(query).show()

In [ ]:
# SQL query to analyze the impact of interest rates on default rates
query = """
SELECT InterestRate, AVG(InterestRate) AS AvgInterestRate, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY InterestRate
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

In [ ]:
# Number of Credit Lines VS Default
query = """
SELECT NumCreditLines, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY NumCreditLines
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

In [ ]:
# Loan Term VS Default
query = """
SELECT LoanTerm, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY LoanTerm
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

In [ ]:
# Converting PySpark DataFrame to Pandas DataFrame
df = spark.sql("SELECT * FROM bank_table")
bank_df = df.toPandas()

In [ ]:
spark.stop()

In [ ]:
default_records_df = pd.DataFrame(bank_df[(bank_df['Default']==1)])
non_default_records_df = pd.DataFrame(bank_df[(bank_df['Default']!=1)])

avg_default_dti = default_records_df['DTIRatio'].mean().__round__(5)
avg_non_default_dti = non_default_records_df['DTIRatio'].mean().__round__(5)

default_credit_lines = default_records_df['NumCreditLines'].mean().__round__(2)
non_default_credit_lines = non_default_records_df['NumCreditLines'].mean().__round__(2)

default_avg_interest = default_records_df['InterestRate'].mean().__round__(3)
non_default_avg_interest = non_default_records_df['InterestRate'].mean().__round__(3)

default_avg_income = default_records_df['Income'].mean().__round__(2)
non_default_avg_income = non_default_records_df['Income'].mean().__round__(2)


print(f'''Of the records for which the loan was defaulted, the average debt to income ratio is: {avg_default_dti}.'''),
print(f'''Of the records for which the loan was defaulted, the average interest rate is: {default_avg_interest}%.'''),
print(f'''Of the records for which the loan was defaulted, the average number of credit lines is: {default_credit_lines}.'''),
print(f'''Of the records for which the loan was defaulted, the average income is: ${default_avg_income}.'''),

print("  "),

print(f'''Of the records for which the loan did NOT default, the average debt to income ratio is: {avg_non_default_dti}.'''),
print(f'''Of the records for which the loan did NOT default the average interest rate is: {non_default_avg_interest}%.'''),
print(f'''Of the records for which the loan did NOT default, the average number of credit lines is: {non_default_credit_lines}.'''),
print(f'''Of the records for which the loan did NOT default, the average income is: ${non_default_avg_income}.''')

In [ ]:
default_records_df.to_csv("default_records.csv")
non_default_records_df.to_csv("non_default_records.csv")

In [ ]:
plot_1 = px.scatter(x=default_records_df['Income'], y=default_records_df['CreditScore'],
                    labels={'x':'Income',
                            'y':'Credit Score'},
                    title="Income vs Credit Score for Defaulted Loans")
plot_1.update_traces(marker_size=2)

In [ ]:

#requires pip install -U kaleido
output_directory = 'Images'
os.makedirs(output_directory, exist_ok=True)
plot_1.write_image('Images/Defaulted_Income_vs_Credit.png')

In [ ]:
plot_2 = px.scatter(x=default_records_df['Income'], y=default_records_df['InterestRate'],
                    labels={'x':'Income',
                            'y':'Interest Rate'},
                    title="Income vs Interest Rate for Defaulted Loans")
plot_2.update_traces(marker_size=2)

In [ ]:
plot_2.write_image('Images/Defaulted_Income_vs_Interest_Rate.png')

In [ ]:
plot_3 = px.scatter(x=non_default_records_df['Income'], y=non_default_records_df['CreditScore'],
                    labels={'x':'Income',
                            'y':'Credit Score'},
                    title="Non-Defaulted Loans Income vs Credit Score")
plot_3.update_traces(marker_size=1)

In [ ]:
plot_3.write_image('Images/non_Defaulted_Income_vs_credit_score.png')

In [ ]:
plot_4 = px.scatter(x=non_default_records_df['Income'], y=non_default_records_df['InterestRate'],
                    labels={'x':'Income',
                            'y':'Interest Rate'},
                    title="Non-Defaulted Loans Income vs Interest Rate")
plot_4.update_traces(marker_size=1)

In [ ]:
plot_4.write_image('Images/non_Defaulted_Income_vs_interest_rate.png')

### Machine Learning/Deep Learning

In [ ]:
# Adding an index column (#)
bank_df.insert(0,'#',range(len(bank_df)))
bank_df.head()

In [ ]:
# Drop the non-beneficial ID columns
bank_df = bank_df.drop(columns=['Education','EmploymentType','MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner'])
bank_df

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = "D:\\downl\\final\\Loan_default.csv"
data = pd.read_csv(file_path)

# Drop the LoanID column as it is not needed for modeling
data = data.drop(columns=['LoanID'])

# Convert categorical variables to numerical
data = pd.get_dummies(data, drop_first=True)

# Separate features and target variable
X = data.drop(columns=['Default'])
y = data['Default']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Create a random forest classifier
model = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=150)

# Train the model on the resampled data
model.fit(X_resampled, y_resampled)





In [ ]:
# Train the model on the resampled data
model.fit(X_resampled, y_resampled)


In [ ]:

# Make predictions on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_pred_proba > 0.7).astype(int)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_adjusted)
report = classification_report(y_test, y_pred_adjusted)

In [ ]:
# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

# Plot feature importances
feature_importance = model.feature_importances_
sorted = feature_importance.argsort()[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance[sorted], align='center')
plt.yticks(range(len(X.columns)), X.columns[sorted])
plt.xlabel('Influence')
plt.title('Feature Importance')
plt.show()



print("Model finished.")